In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr

import matplotlib.pyplot as plt
import numpy as np
import argparse
import subprocess
import os
from scipy.spatial import cKDTree

from netCDF4 import default_fillvals

fillval_f = default_fillvals['f8']
fillval_i = default_fillvals['i4']

In [8]:
direc = '/Users/diana/Dropbox/UW/Research/rasm/data/25km'
restart_50km_filename = 'vic.r.r31.20120730.09-01.nc'
ds = xr.open_dataset(os.path.join(direc, restart_50km_filename))

In [4]:
domain_25km_filename = 'domain.lnd.wr25b_ar9v4.170413.nc'
direc = '/Users/diana/Dropbox/UW/Research/rasm/data/25km'
domain_25km = xr.open_dataset(os.path.join(direc, domain_25km_filename))

lon_25km = domain_25km['xc'].values
lat_25km = domain_25km['yc'].values
mask_25km = domain_25km['mask'].values

domain_50km_filename = 'domain.lnd.wr50a_ar9v4.100920.nc'
domain_50km = xr.open_dataset(os.path.join(direc, domain_50km_filename))

lon_50km = domain_50km['xc'].values
lat_50km = domain_50km['yc'].values
mask_50km = domain_50km['mask'].values

In [5]:
def lon_lat_to_cartesian(lon, lat, R = 1):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x = R * np.cos(lat_r) * np.cos(lon_r)
    y = R * np.cos(lat_r) * np.sin(lon_r)
    z = R * np.sin(lat_r)

    return x, y, z

In [6]:
# lon, lat from coarser res data
xs, ys, zs = lon_lat_to_cartesian(lon_50km.flatten(), lat_50km.flatten()) 

# lon, lat from higher res data
xt, yt, zt = lon_lat_to_cartesian(lon_25km.flatten(), lat_25km.flatten())

# build KD tree
zipped_50km = np.dstack(([xs, ys, zs]))[0]
tree = cKDTree(zipped_50km)

# find indices of the nearest neighbors in the flattened array 
zipped_25km = np.dstack(([xt, yt, zt]))[0]
d, inds = tree.query(zipped_25km, k=1)

In [28]:
restart_ds = xr.Dataset()
###########################################
#################################### vegetat_type_num #################################

nearest_vegnum = ds['vegetat_type_num'].values[inds]
restart_ds['vegetat_type_num'] = xr.DataArray(nearest_vegnum, 
                                              dims=('IX'), encoding={'dtype': 'int32', 
                                                                     '_FillValue': fillval_i})

#################################### STILL_STORM #################################

nearest_stillstorm = ds['STILL_STORM'].values[inds]
restart_ds['STILL_STORM'] = xr.DataArray(nearest_stillstorm, 
                                              dims=('IX'),
                                         encoding={'dtype': 'b', '_FillValue': 0})

#################################### DRY_TIME #################################

nearest_drytime = ds['DRY_TIME'].values[inds]
restart_ds['DRY_TIME'] = xr.DataArray(nearest_drytime, 
                                              dims=('IX'))

#################################### dz_node #################################

nearest_dznode = ds['dz_node'].values[:, inds]
restart_ds['dz_node'] = xr.DataArray(nearest_dznode, 
                                              dims=('Nnode', 'IX'))

#################################### mu #################################

nearest_mu = ds['mu'].values[:, inds]
restart_ds['mu'] = xr.DataArray(nearest_mu, 
                                dims=('Nveg','IX'))

#################################### moist #################################

nearest_moist = ds['moist'].values[:, :, :, :, inds]

restart_ds['moist'] = xr.DataArray(nearest_moist, 
                                dims=('Nlayer', 'Ndist', 'Nband', 'Nveg', 'IX'))

#################################### ice #################################

nearest_ice = ds['ice'].values[:, :, :, :, inds]
restart_ds['ice'] = xr.DataArray(nearest_ice,
                                dims=('Nlayer', 'Ndist', 'Nband', 'Nveg', 'IX'))

#################################### Wdew #################################

nearest_wdew = ds['Wdew'].values[:, :, :, inds]
restart_ds['Wdew'] = xr.DataArray(nearest_wdew,
                                  dims=('Ndist', 'Nband', 'Nveg', 'IX'))

#################################### last_snow #################################

nearest_lastsnow = ds['last_snow'].values[:, :, inds]
restart_ds['last_snow'] = xr.DataArray(nearest_lastsnow, 
                                       dims=('Nband', 'Nveg', 'IX'), 
                                       encoding={'dtype': 'int32', '_FillValue': fillval_i})

#################################### MELTING #################################

nearest_melting = ds['MELTING'].values[:, :, inds]
restart_ds['MELTING'] = xr.DataArray(nearest_melting, 
                                       encoding={'dtype': 'b', '_FillValue': 0},
                                       dims=('Nband', 'Nveg', 'IX'))
                                       
#################################### coverage #################################
nearest_coverage = ds['coverage'].values[:, :, inds]
restart_ds['coverage'] = xr.DataArray(nearest_coverage, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### swq #################################

nearest_swq = ds['swq'].values[:, :, inds]
restart_ds['swq'] = xr.DataArray(nearest_swq, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### surf_temp #################################

nearest_surftemp = ds['surf_temp'].values[:, :, inds]
restart_ds['surf_temp'] = xr.DataArray(nearest_surftemp, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### surf_water #################################

nearest_surfwater = ds['surf_water'].values[:, :, inds]
restart_ds['surf_water'] = xr.DataArray(nearest_surfwater, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### pack_temp #################################

nearest_packtemp = ds['pack_temp'].values[:, :, inds]
restart_ds['pack_temp'] = xr.DataArray(nearest_packtemp, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### pack_water #################################

nearest_packwater = ds['pack_water'].values[:, :, inds]
restart_ds['pack_water'] = xr.DataArray(nearest_packwater, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### density #################################

nearest_density = ds['density'].values[:, :, inds]
restart_ds['density'] = xr.DataArray(nearest_density, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### coldcontent #################################

nearest_coldcontent = ds['coldcontent'].values[:, :, inds]
restart_ds['coldcontent'] = xr.DataArray(nearest_coldcontent, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### snow_canopy #################################

nearest_snowcanopy = ds['snow_canopy'].values[:, :, inds]
restart_ds['snow_canopy'] = xr.DataArray(nearest_snowcanopy, 
                                       dims=('Nband', 'Nveg', 'IX'))

#################################### T #################################

nearest_t = ds['T'].values[:, :, :, inds]
restart_ds['T'] = xr.DataArray(nearest_t, 
                                  dims=('Nnode', 'Nband', 'Nveg', 'IX'))

In [29]:
restart_ds.rename({'Nband': 'Nbands'}, inplace=True)
# vars to encode as ints: vegetat_type_num, STILL_STORM, 
# restart_ds.transpose(('Nbands', 'Nlayer', 'Ndist', 'Nnode', 'Nveg', 'IX'))
direc = '/Users/diana/Dropbox/UW/Research/rasm/data/25km'
restart_ds.to_netcdf(os.path.join(direc, 
                                  'restart_25km_20190329.nc'), 
                     format='NETCDF4_CLASSIC')